<a href="https://colab.research.google.com/github/RuchitaSuranagi/-Skin-Cancer-Classifier/blob/main/multi_class_skin_lesions_DP_Opacus_Skin_Cancer_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***DP-Opacus-Skin-Cancer-Classification_AITD_Batch_22***

Detecting the type of skin cancer by training the model with 7 type of skin lesions

# Skin Cancer Classifier

In [ ]:
#Import the necessary modules
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
import torchcsprng as csprng
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader,Dataset

import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from PIL import Image
import os,cv2,itertools
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

In [ ]:
!git clone https://github.com/pytorch/opacus.git

Cloning into 'opacus'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (301/301), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 2202 (delta 163), reused 197 (delta 81), pack-reused 1901
Receiving objects: 100% (2202/2202), 1.59 MiB | 11.52 MiB/s, done.
Resolving deltas: 100% (1258/1258), done.


In [ ]:
import os
os.chdir('opacus')

In [ ]:
!pip install -e .

Obtaining file:///content/opacus
  Running setup.py develop for opacus


In [ ]:
from opacus import PrivacyEngine
from opacus.utils import module_modification

Note: The HAM10000 dataset was obtained from- https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T 

The following types of skin cancer can be predicated by the model
- MEL: “Melanoma”
-NV: “Melanocytic nevus”
-BCC: “Basal cell carcinoma”
-AKIEC: “Actinic keratosis / Bowen’s disease (intraepithelial carcinoma)”
-BKL: “Benign keratosis (solar lentigo / seborrheic keratosis / lichen planus-like keratosis)”
-DF: “Dermatofibroma”
-VASC: “Vascular lesion”

In [ ]:
# authorize access to google drive

from google.colab import drive
#drive.mount("/content/drive", force_remount=True)
drive.mount('/content/drive')
!pwd

Mounted at /content/drive
/content/opacus


In [ ]:
%cd "/content/drive/My Drive/Project-3rd year/data"

/content/drive/My Drive/Project-3rd year/data


In [ ]:
#There are 7 types of classes in the dataset for lesions as specified:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'melenoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [ ]:
data_dir = '/content/drive/My Drive/Project-3rd year/data/'
all_image_path = glob(os.path.join(data_dir, '*', '*.jpg'))
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}

In [ ]:
def compute_img_mean_std(image_paths):
#computing the mean and std of three channel on the whole dataset,
#first we should normalize the image


    img_h, img_w = 224, 224
    imgs = []
    NORM_MEAN, NORM_STD_DEV = [], []

    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        imgs.append(img)

    imgs = np.stack(imgs, axis=3)
    print(imgs.shape)

    imgs = imgs.astype(np.float32) / 255.

    for i in range(3):
        pixels = imgs[:, :, i, :].ravel()  # resize to one row
        NORM_MEAN.append(np.mean(pixels))
        NORM_STD_DEV.append(np.std(pixels))

    NORM_MEAN.reverse()  # BGR --> RGB
    NORM_STD_DEV.reverse()

    print("NORM_MEAN = {}".format(NORM_MEAN))
    print("NORM_STD_DEV = {}".format(NORM_STD_DEV))
    return NORM_MEAN,NORM_STD_DEV

In [ ]:
NORM_MEAN,NORM_STD_DEV = compute_img_mean_std(all_image_path)

100%|██████████| 10015/10015 [56:06<00:00,  2.98it/s]


(224, 224, 3, 10015)
NORM_MEAN = [0.7637123, 0.53815764, 0.56163645]
NORM_STD_DEV = [0.13791, 0.1587932, 0.17676742]


In [ ]:
#Hyper-parameters

MAX_GRAD_NORM = 1.2  #.5 #1.5 or1.0
NOISE_MULTIPLIER = .38 #1.3 or .38
DELTA = 1e-5

LR = 1e-3
NUM_WORKERS = 2

BATCH_SIZE = 32
VIRTUAL_BATCH_SIZE = 64

In [ ]:
#Reading the data from the csv file and saving it as a dataframe.
DF = pd.read_csv('/content/drive/My Drive/Project-3rd year/data/HAM10000_metadata.csv')

#Adding columns to the original DataFrame, dx_name(the whole name), dx_id(the corresponding index of cell type, as the image label).
DF['dx_name'] = DF['dx'].map(lesion_type_dict.get)
DF['dx_id'] = pd.Categorical(DF['dx_name']).codes 

#Converting each lesion into  a numerical code.
DF.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dx_name,dx_id
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,Benign keratosis-like lesions,2


In [ ]:
DF.shape

(10015, 9)

Training dataset and a testing dataset.
Check for duplicates in our dataset as those are not required for test set.

In [ ]:
#Creating a new dataframe df_undup that contains only the non-duplicate elements.
DF_NO_DUBLICATE = DF.groupby('lesion_id').count()

#Filters out so that we have only one image associated with each lesion_id
DF_NO_DUBLICATE = DF_NO_DUBLICATE[DF_NO_DUBLICATE['image_id'] == 1]
DF_NO_DUBLICATE.reset_index(inplace=True)
DF_NO_DUBLICATE.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dx_name,dx_id
0,HAM_0000001,1,1,1,1,1,1,1,1
1,HAM_0000003,1,1,1,1,1,1,1,1
2,HAM_0000004,1,1,1,1,1,1,1,1
3,HAM_0000007,1,1,1,1,1,1,1,1
4,HAM_0000008,1,1,1,1,1,1,1,1


In [ ]:
#creating new column that specifies whether lesion_id is duplicated or not.

def get_duplicates(x):
    unique_list = list(DF_NO_DUBLICATE['lesion_id'])
    if x in unique_list:
        return 'unique'
    else:
        return 'duplicated'
DF['duplicates'] = DF['lesion_id']
DF['duplicates'] = DF['duplicates'].apply(get_duplicates)
DF.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dx_name,dx_id,duplicates
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2,duplicated
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2,duplicated
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2,duplicated
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2,duplicated
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,Benign keratosis-like lesions,2,duplicated


In [ ]:
# number of duplicates.
DF['duplicates'].value_counts()

unique        5514
duplicated    4501
Name: duplicates, dtype: int64

In [ ]:
#creating the dataframe with only non-duplicate elements.
DF_NO_DUBLICATE = DF[DF['duplicates'] == 'unique']
DF_NO_DUBLICATE.shape

(5514, 10)

In [ ]:
#Creation of a validation set by randomly choosing 20% data from orignal DF and 36.3% rows from non-duplicate dataset.

y = DF_NO_DUBLICATE['dx_id']
_, DF_TEST = train_test_split(DF_NO_DUBLICATE, test_size=0.363, random_state=101, stratify=y)
DF_TEST.shape

(2002, 10)

Now we are required to create our training dataset which can contain the duplicate elements but non the validation ones.

In [ ]:
#To check whether a row is present in validation or not.
def get_val_rows(x): 
    val_list = list(DF_TEST['image_id'])
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

# create a new colum that is a copy of the image_id column
DF['train_or_val'] = DF['image_id']

# apply the function to this new column
DF['train_or_val'] = DF['train_or_val'].apply(get_val_rows)

# filter out train rows

DF_TRAIN = DF[DF['train_or_val'] == 'train']
print(len(DF_TRAIN))

8013


In [ ]:
DF_TRAIN.shape

(8013, 11)

In [ ]:
print(DF_TRAIN['dx_name'].value_counts(), "\n")
print(DF_TRAIN['dx_id'].value_counts())

Melanocytic nevi                  5102
melenoma                          1030
Benign keratosis-like lesions      939
Basal cell carcinoma               450
Actinic keratoses                  272
Vascular lesions                   119
Dermatofibroma                     101
Name: dx_name, dtype: int64 

4    5102
6    1030
2     939
1     450
0     272
5     119
3     101
Name: dx_id, dtype: int64


In [ ]:
print(DF_TEST['dx_name'].value_counts(), "\n")
print(DF_TEST['dx_id'].value_counts())

Melanocytic nevi                  1603
Benign keratosis-like lesions      160
melenoma                            83
Basal cell carcinoma                64
Actinic keratoses                   55
Vascular lesions                    23
Dermatofibroma                      14
Name: dx_name, dtype: int64 

4    1603
2     160
6      83
1      64
0      55
5      23
3      14
Name: dx_id, dtype: int64


Since train dataset is very skewed  and unbalanced, so we augment it to even it out and have a uniform distribution.

In [ ]:
#Setting augmentation rates according to ratio of examples in dataset.

data_aug_rate = [15,10,5,50,0,40,5] 

for i in range(7):
    if data_aug_rate[i]:
        DF_TRAIN=DF_TRAIN.append([DF_TRAIN.loc[DF_TRAIN['dx_id'] == i,:]]*(data_aug_rate[i]-1), ignore_index=True)
DF_TRAIN['dx_id'].value_counts()

6    5150
4    5102
3    5050
5    4760
2    4695
1    4500
0    4080
Name: dx_id, dtype: int64

So now we have got our training dataset with evenly distributed data.

In [ ]:
DF_ = DF_TRAIN.reset_index()
DF_TEST = DF_TEST.reset_index()

### Creating train and testing Dataloaders.

In [ ]:
#The mean and standard values of images for normalisation with data augmentations  

train_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),   
    transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(NORM_MEAN, NORM_STD_DEV)
])

In [ ]:

test_transforms = transforms.Compose([transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(NORM_MEAN, NORM_STD_DEV)])

In [ ]:
#class HAM10000 lets image and its associated label be combined into a single dataset.
class HAM10000(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load data and get label
        a=self.df['image_id'][index]
        images = Image.open("/content/drive/My Drive/Project-3rd year/data/train/" +a+'.jpg')
        labels= torch.tensor(int(self.df['dx_id'][index]))

        if self.transform:
            images = self.transform(images)

        return images, labels

In [ ]:
#Creating the training dataset.

train_dataset = HAM10000(DF_TRAIN, transform=train_transforms)
train_dataset.__getitem__(0)

(tensor([[[-5.5378, -5.5378, -5.5378,  ..., -5.5378, -5.5378, -5.5378],
          [-5.5378, -5.5378, -5.5378,  ..., -5.5378, -5.5378, -5.5378],
          [-5.5378, -5.5378, -5.5378,  ..., -5.5378, -5.5378, -5.5378],
          ...,
          [-5.5378, -5.5378, -5.5378,  ..., -5.5378, -5.5378, -5.5378],
          [-5.5378, -5.5378, -5.5378,  ..., -5.5378, -5.5378, -5.5378],
          [-5.5378, -5.5378, -5.5378,  ..., -5.5378, -5.5378, -5.5378]],
 
         [[-3.3890, -3.3890, -3.3890,  ..., -3.3890, -3.3890, -3.3890],
          [-3.3890, -3.3890, -3.3890,  ..., -3.3890, -3.3890, -3.3890],
          [-3.3890, -3.3890, -3.3890,  ..., -3.3890, -3.3890, -3.3890],
          ...,
          [-3.3890, -3.3890, -3.3890,  ..., -3.3890, -3.3890, -3.3890],
          [-3.3890, -3.3890, -3.3890,  ..., -3.3890, -3.3890, -3.3890],
          [-3.3890, -3.3890, -3.3890,  ..., -3.3890, -3.3890, -3.3890]],
 
         [[-3.1773, -3.1773, -3.1773,  ..., -3.1773, -3.1773, -3.1773],
          [-3.1773, -3.1773,

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=NUM_WORKERS, drop_last = True,)

In [ ]:
test_dataset = HAM10000(DF_TEST , transform = test_transforms)
test_dataset.__getitem__(0)

(tensor([[[ 0.6328,  0.5475,  0.3769,  ...,  0.8034,  0.8034,  0.6328],
          [ 0.7750,  0.7465,  0.5190,  ...,  0.8887,  0.9740,  0.5759],
          [ 0.7465,  0.7181,  0.8318,  ...,  0.8887,  0.7750,  0.9171],
          ...,
          [ 0.4622,  0.5190,  0.3769,  ...,  0.5759,  0.6612,  0.6612],
          [ 0.3484,  0.3200,  0.4906,  ...,  0.5190,  0.6897,  0.6328],
          [ 0.3769,  0.3200,  0.3769,  ...,  0.6044,  0.4906,  0.5190]],
 
         [[-0.2773, -0.4749, -0.7219,  ...,  0.1672,  0.1425, -0.0551],
          [-0.2279, -0.3020, -0.5490,  ...,  0.1672,  0.1672, -0.2279],
          [-0.2773, -0.3020, -0.1786,  ...,  0.0684, -0.0798, -0.0551],
          ...,
          [-0.2526, -0.1292, -0.2033,  ..., -0.0551,  0.0684,  0.0684],
          [-0.2773, -0.2526, -0.1045,  ..., -0.0798,  0.0684,  0.0190],
          [-0.1786, -0.2526, -0.2526,  ..., -0.0551, -0.1539, -0.0798]],
 
         [[-0.4485, -0.5373, -0.7813,  ...,  0.0839,  0.0617, -0.1379],
          [-0.3154, -0.3598,

In [ ]:
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=BATCH_SIZE, shuffle=False,num_workers=NUM_WORKERS,drop_last = True,)

## Model

In [ ]:
from torchvision import models

model = models.resnet18(num_classes=7)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
from opacus.dp_model_inspector import DPModelInspector

inspector = DPModelInspector()

In [ ]:
#For model to work with Opacus we need to replace all the BatchNorm layers with GroupNorm using the convert_batchnorm_modules util function, as BatchNorm layers are not supported.

from opacus.utils import module_modification

model = module_modification.convert_batchnorm_modules(model)
inspector = DPModelInspector()
print(f"Is the model valid? {inspector.validate(model)}")

Is the model valid? True


In [ ]:
#Checking if the machine supports cuda.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
device

device(type='cuda')

In [ ]:
import torch.nn as nn
import torch.optim as optim
# using RMSprop as Opacus’ privacy engine can attach to any (first-order) optimizer.

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=LR)

## Training

In [ ]:
def accuracy(preds, labels):
    return (preds == labels).mean()

In [ ]:
from opacus import PrivacyEngine
print(f"Using sigma={NOISE_MULTIPLIER} and C={MAX_GRAD_NORM}")

privacy_engine = PrivacyEngine(
    model,
    batch_size=VIRTUAL_BATCH_SIZE,
    sample_size=len(train_dataset),
    alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64)),
    noise_multiplier=NOISE_MULTIPLIER,
    max_grad_norm=MAX_GRAD_NORM,
)
privacy_engine.attach(optimizer)


Using sigma=0.38 and C=1.2


/content/opacus/opacus/privacy_engine.py:104: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


In [ ]:
# VIRTUAL_BATCH_SIZE should be divisible by BATCH_SIZE

assert VIRTUAL_BATCH_SIZE % BATCH_SIZE == 0 
virtual_batch_rate = int(VIRTUAL_BATCH_SIZE / BATCH_SIZE)

In [ ]:
def train(model, train_loader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []

    for i, (images, target) in enumerate(train_loader):        
        images = images.to(device)
        target = target.to(device)


        # compute output
        output = model(images)
        loss = criterion(output, target)
        
        preds = np.argmax(output.detach().cpu().numpy(), axis=1)
        labels = target.detach().cpu().numpy()
        
        # measure accuracy and record loss
        acc = accuracy(preds, labels)

        losses.append(loss.item())
        top1_acc.append(acc)
        
        loss.backward()
        	
        # take a real optimizer step after N_VIRTUAL_STEP steps t
        if ((i + 1) % virtual_batch_rate == 0) or ((i + 1) == len(train_loader)):
            optimizer.step()
            optimizer.zero_grad()
        else:
            optimizer.virtual_step() # take a virtual step

        if i % 200 == 0:
            epsilon, best_alpha = optimizer.privacy_engine.get_privacy_spent(DELTA)
            print(
                f"\tTrain Epoch: {epoch} \t"
                f"Loss: {np.mean(losses):.6f} "
                f"Acc: {np.mean(top1_acc) * 100:.6f} "
                f"(ε = {epsilon:.2f}, δ = {DELTA})"
                 )


In [ ]:
def test(model, test_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)
    
    print(
        f"\tTest set:"
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)


## Train the network

In [ ]:
from tqdm.notebook import tqdm_notebook

In [ ]:
for epoch in tqdm_notebook(range(20), desc="Epoch", unit="epoch"):
    train(model, train_loader, optimizer, epoch + 1, device)

	Train Epoch: 1 	Loss: 2.052257 Acc: 18.750000 (ε = 0.19, δ = 1e-05)
	Train Epoch: 1 	Loss: 1.977730 Acc: 19.838308 (ε = 10.26, δ = 1e-05)
	Train Epoch: 1 	Loss: 1.894244 Acc: 22.708853 (ε = 11.29, δ = 1e-05)
	Train Epoch: 1 	Loss: 1.841438 Acc: 24.828411 (ε = 12.11, δ = 1e-05)
	Train Epoch: 1 	Loss: 1.799706 Acc: 26.814139 (ε = 12.66, δ = 1e-05)
	Train Epoch: 1 	Loss: 1.759468 Acc: 28.565185 (ε = 13.20, δ = 1e-05)


/content/opacus/opacus/privacy_engine.py:280: UserWarning: PrivacyEngine expected a batch of size 64 but the last step received a batch of size 32. This means that the privacy analysis will be a bit more pessimistic. You can set `drop_last = True` in your PyTorch dataloader to avoid this problem completely
  f"PrivacyEngine expected a batch of size {self.batch_size} "


	Train Epoch: 2 	Loss: 1.672971 Acc: 31.250000 (ε = 13.32, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.534559 Acc: 39.194652 (ε = 13.84, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.532129 Acc: 39.339152 (ε = 14.21, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.519919 Acc: 40.229825 (ε = 14.58, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.507435 Acc: 40.718633 (ε = 14.96, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.504585 Acc: 41.243132 (ε = 15.33, δ = 1e-05)
	Train Epoch: 3 	Loss: 1.499317 Acc: 50.000000 (ε = 15.41, δ = 1e-05)
	Train Epoch: 3 	Loss: 1.461239 Acc: 44.263060 (ε = 15.78, δ = 1e-05)
	Train Epoch: 3 	Loss: 1.460096 Acc: 45.129364 (ε = 16.12, δ = 1e-05)
	Train Epoch: 3 	Loss: 1.462763 Acc: 45.179908 (ε = 16.39, δ = 1e-05)
	Train Epoch: 3 	Loss: 1.462467 Acc: 45.564139 (ε = 16.66, δ = 1e-05)
	Train Epoch: 3 	Loss: 1.467837 Acc: 45.610639 (ε = 16.93, δ = 1e-05)
	Train Epoch: 4 	Loss: 1.533720 Acc: 43.750000 (ε = 16.98, δ = 1e-05)
	Train Epoch: 4 	Loss: 1.475475 Acc: 47.108209 (ε = 17.25, δ = 1e-05)
	Train Epoch: 4 	Los

In [ ]:
top1_acc = test(model, test_loader, device)

	Test set:Loss: 0.883671 Acc: 75.252016 


Training a **private ResNet18 model** for **20 epochs**.

**Test Accuracy** was found to be **75.252016** with 
**Epsilon (ε) = 31.96,Epsilon** which is proportional to the privacy budget.


We trained a non-private ResNet18 model for 20 epochs using the same hyper-parameters as above and with BatchNorm replaced with GroupNorm. The results of that was as following

Test set:Loss: 0.677344,    Acc: 81.955645 

That is, **Test Accuracy** was found to be **81.955645** 

Link for the non-private ResNet18 model : https://colab.research.google.com/drive/1eb840oBGUpPvi7niMy990b5mrbUJCCY8?usp=sharing 